In [21]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
from PIL import Image

In [23]:
# Location of repo's / files
# label_file = '../Data/Development/inliers/male_casual/labels.csv'
# label_file = '../Data/Development/inliers/female_business/labels.csv'
# dataset = '../Data/Development/inliers/' 


label_file = '../Data/non-pedestrian/images/cylinder/labels.csv'
dataset = '../Data/non-pedestrian/' 

# Read in data, and print example's 
df = pd.read_csv(label_file)
df.head()

,file,frame_index,image_width,image_height,x_min,x_max,y_min,y_max,is_occluded,current_distance,class_text,run_id,scenario_type,object_type,start_distance_from_car,speed,angle,offset_from_road_center
0,qFaN520H78j9p7FUN0gv5/cam000001.png,1,752,480,NaN,NaN,NaN,NaN,False,12.68,background,qFaN520H78j9p7FUN0gv5,left,cylinder,10,4,90,NaN
1,qFaN520H78j9p7FUN0gv5/cam000002.png,2,752,480,NaN,NaN,NaN,NaN,False,12.63,background,qFaN520H78j9p7FUN0gv5,left,cylinder,10,4,90,NaN
2,qFaN520H78j9p7FUN0gv5/cam000003.png,3,752,480,NaN,NaN,NaN,NaN,False,12.54,background,qFaN520H78j9p7FUN0gv5,left,cylinder,10,4,90,NaN
3,qFaN520H78j9p7FUN0gv5/cam000004.png,4,752,480,NaN,NaN,NaN,NaN,False,12.42,background,qFaN520H78j9p7FUN0gv5,left,cylinder,10,4,90,NaN
4,qFaN520H78j9p7FUN0gv5/cam000005.png,5,752,480,NaN,NaN,NaN,NaN,False,12.28,background,qFaN520H78j9p7FUN0gv5,left,cylinder,10,4,90,NaN


In [24]:
import tqdm.notebook as tq

In [25]:
dataset_name = 'vae_outliers'
if not os.path.isdir(dataset_name): 
    os.mkdir(dataset_name)

In [26]:
def find_dimension(val):
    x = 64
    while (val // x > 0) and x < 256: x*=2
    return x

In [32]:
import time 
for index, row in tq.tqdm(df.iterrows(), total=df.shape[0]):
    parts = row['file'].replace('\\','/').split('/')
    path = os.path.join(dataset_name, 'cylinder')
    
    if not os.path.isdir(path): 
        os.mkdir(path)
        
    for part in parts:
        path = os.path.join(path, part)
        if not os.path.isdir(path) and not path.endswith('.png'):
            os.mkdir(path)
    
        # Extract BB information
    bounding_box = row.loc[(['x_min','x_max','y_min','y_max'])]
    
    # Check if they're NaN --> Take next if true. 
    if bounding_box.isnull().values.any(): 
        continue 
    else: 
        # Computing Label position 
        xmin, xmax, ymin, ymax = bounding_box.values
        x_centre = ((xmin+xmax)//2) 
        y_centre = ((ymin+ymax)//2) 
        width = 128
        height = 128 
    
        # Loading image 
        img_path = os.path.join(dataset, 'images', 'cylinder', parts[0])
        for part in parts[1:]:
            img_path = os.path.join(img_path, part)
        
        im = Image.open(img_path)
        img_width, img_height = im.size 
        
        
        # Setting the points for cropped image
        left = x_centre - width
        top = y_centre - height
        right = x_centre + width
        bottom = y_centre + height 
        
        # Correct crop if it is out of bounds
        if left < 0: 
            right = right - left 
            left = 0
        elif right > img_width: 
            left = left - (right - img_width)
            right = img_width
        
        if top < 0: 
            bottom = bottom - top 
            top = 0
        elif bottom > img_height: 
            top = top - (bottom - img_height)
            bottom = img_height

        # Save image 
        im1 = im.crop((left, top, right, bottom))
        im1.save(path)


In [26]:
os.path.join(path)

'vae_dataset\\male_casual\\iCg8fMTLyEahuVnQ0Hp2C\\cam000027.png'

In [15]:
dataset

'../Data/Development/images'

In [13]:
parts = row['file'].replace('\\','/').split('/')
path = os.path.join(dataset, parts[0], 'images')
for part in parts[1:]:
    path = os.path.join(path, part)
print(path)

../Data/Development/inliers/female_business\images\VpFfsBKEtFllhmpRK9mFv\cam000001.png
